In [4]:
import pandas as pd

In [6]:
print ("test something here")


file_list=[]

for input in range(2006,2019):
    file_list.append(pd.read_csv("/Users/matthiasweber/outputEXIFcsvs/PhotoEXIFDataDayType_%s.csv"%(str(input))))
    
for file in file_list:
    print (file_list)
    

test something here
[                                            SourceFile  \
0    /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
1    /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
2    /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
3    /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
4    /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
..                                                 ...   
571  /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
572  /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
573  /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
574  /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
575  /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   

               DateTimeOriginal FocalLength ExposureTime    Model  ISO  
0    Jul 10 Mon ,2006, 12:36:06      6.3 mm        1/800  SP500UZ   80  
1    Jul 10 Mon ,2006, 16:52:49      6.3 mm        1/320  SP500UZ   80  
2    Jul 10 Mon ,2006, 17:17:26      6.3 mm        1/640  SP500

[                                            SourceFile  \
0    /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
1    /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
2    /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
3    /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
4    /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
..                                                 ...   
571  /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
572  /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
573  /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
574  /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   
575  /Volumes/INTENSO/Backups.backupdb/Matthias’s M...   

               DateTimeOriginal FocalLength ExposureTime    Model  ISO  
0    Jul 10 Mon ,2006, 12:36:06      6.3 mm        1/800  SP500UZ   80  
1    Jul 10 Mon ,2006, 16:52:49      6.3 mm        1/320  SP500UZ   80  
2    Jul 10 Mon ,2006, 17:17:26      6.3 mm        1/640  SP500UZ   80  
3    Jul 1